In [1]:
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification
from transformers import AutoFeatureExtractor, AutoModelForImageClassification
from PIL import Image
import cv2
import os
import pathlib
import shutil
import requests


In [5]:

model_id = "locih/violence_classification"
# Load the model and feature extractor
model_violence = ViTForImageClassification.from_pretrained(model_id)
extractor_violence = ViTFeatureExtractor.from_pretrained(model_id)

model_name = "rizvandwiki/gender-classification-2"
extractor_gender = AutoFeatureExtractor.from_pretrained(model_name)
model_gender = AutoModelForImageClassification.from_pretrained(model_name)

C:\Users\sagni\AppData\Roaming\Python\Python312\site-packages\transformers\models\vit\feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
directory_to_remove = "op"

# Check if the directory exists before attempting to remove it
if os.path.exists(directory_to_remove) and os.path.isdir(directory_to_remove):
    try:
        shutil.rmtree(directory_to_remove)
        print(f"Directory '{directory_to_remove}' and its contents removed successfully.")
    except OSError as e:
        print(f"Error: {directory_to_remove} : {e.strerror}")
else:
    print(f"Directory '{directory_to_remove}' does not exist or is not a directory.")

In [9]:
video_path = "violent/cam1/28.mp4" #37 violent

In [ ]:

def extract_and_save_frames(video_path, save_dir="op", num_frames=15):
    # Create the save directory if it doesn't exist
    os.makedirs(save_dir, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames == 0:
        raise ValueError("❌ No frames found in the video. Check if the path is correct.")

    print(f"🎞️ Total frames in video: {total_frames}")

    # Determine frame positions to capture
    interval = total_frames // num_frames
    frame_ids = [i * interval for i in range(num_frames)]

    print(f"📸 Capturing frames at positions: {frame_ids}")

    saved_count = 0
    for i in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        if i in frame_ids:
            filename = os.path.join(save_dir, f"frame_{saved_count + 1}.jpg")
            cv2.imwrite(filename, frame)
            print(f"✅ Saved {filename}")
            saved_count += 1

        if saved_count >= num_frames:
            break

    cap.release()
    print(f" Done! {saved_count} frames saved to '{save_dir}'")

# === Example Usage ===
save_dir = "op"
extract_and_save_frames(video_path, save_dir)


In [13]:
def predict_gender(frames):
    gen=[]
    for path in frames.iterdir():
        frame = cv2.imread(path)
        image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        inputs = extractor_gender(images=image, return_tensors="pt")
        with torch.no_grad():
            outputs = model_gender(**inputs)
        
        # Convert logits to predicted label
        predicted_class = outputs.logits.argmax(-1).item()
        label = model_gender.config.id2label[predicted_class]

        gen.append(label)
    return max(set(gen), key=gen.count)

pt = pathlib.Path("op")

def violence_detection(frames):
    l = []
    for i in frames.iterdir():
        image = Image.open(i)
        
        # Preprocess the image
        inputs = extractor_violence(images=image, return_tensors="pt")
        
        # Perform inference
        with torch.no_grad():
            outputs = model_violence(**inputs)
            logits = outputs.logits
            predicted_class_idx = logits.argmax(-1).item()
        
        # Print the predicted class
        l.append(model_violence.config.id2label[predicted_class_idx])
    f = max(set(l), key=l.count)
    if f=='safe':
        return 0,l
    else:
        return 1,l

In [ ]:
predict_gender(pt)

In [ ]:
o,l = violence_detection(pt)
o,l

In [40]:
def create_sos():
    url = "http://localhost:8000/api/create-sos/"

# JSON payload
    payload = {
        "name": "cam001",
        "sos_type": 1,
        "initial_latitude": 22.554450,
        "initial_longitude": 88.349850
    }
    
    # Send POST request
    response = requests.post(url, json=payload)
    res = response.json()
    # Print result
    print("Status Code:", response.status_code)
    print("Response:", response.json())
    return res["sos_id"]




def upload_images():
    # API endpoint
    url = "http://localhost:8000/api/upload-sos-images/"
    
    
    # Form data
    sos_id = create_sos()
    descriptions = ["Evidence photo", "Location photo"]
    image_dir = "op"
    # Open image files
    files = [
    ("images", open(os.path.join(image_dir, filename), "rb"))
    for filename in os.listdir(image_dir)
    if filename.lower().endswith((".jpg", ".jpeg", ".png"))  # Optional: filter by image type
    ]
    
    # Add descriptions (same key, different values)
    data = [("sos_id", str(sos_id))]
    data += [("descriptions", desc) for desc in descriptions]
    
    # Send POST request
    response = requests.post(url, files=files, data=data)
    
    # Print result
    print("Status Code:", response.status_code)
    print("Response:", response.json())

In [ ]:
if o==1:
    upload_images()


In [34]:
def main(video_path = "violent/cam1/87.mp4"):
    directory_to_remove = "op"

    # Check if the directory exists before attempting to remove it
    if os.path.exists(directory_to_remove) and os.path.isdir(directory_to_remove):
        try:
            shutil.rmtree(directory_to_remove)
            print(f"Directory '{directory_to_remove}' and its contents removed successfully.")
        except OSError as e:
            print(f"Error: {directory_to_remove} : {e.strerror}")
    else:
        print(f"Directory '{directory_to_remove}' does not exist or is not a directory.")

    extract_and_save_frames(video_path)
    pt = pathlib.Path("op")
    gender = predict_gender(pt)

    if gender=='female':
        print('females detected, proceeding to violence detection')
        o,l = violence_detection(pt)
        print(l)

        if o==1:
            print("violence detected, sending frames to server")
            upload_images()
    else:
        print('females not detected, exiting')


In [46]:
main("violent/cam1/87.mp4")

Directory 'op' and its contents removed successfully.
🎞️ Total frames in video: 327
📸 Capturing frames at positions: [0, 21, 42, 63, 84, 105, 126, 147, 168, 189, 210, 231, 252, 273, 294]
✅ Saved op\frame_1.jpg
✅ Saved op\frame_2.jpg
✅ Saved op\frame_3.jpg
✅ Saved op\frame_4.jpg
✅ Saved op\frame_5.jpg
✅ Saved op\frame_6.jpg
✅ Saved op\frame_7.jpg
✅ Saved op\frame_8.jpg
✅ Saved op\frame_9.jpg
✅ Saved op\frame_10.jpg
✅ Saved op\frame_11.jpg
✅ Saved op\frame_12.jpg
✅ Saved op\frame_13.jpg
✅ Saved op\frame_14.jpg
✅ Saved op\frame_15.jpg
 Done! 15 frames saved to 'op'
females detected, proceeding to violence detection
['unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'unsafe', 'safe', 'unsafe', 'safe', 'safe', 'safe', 'safe', 'safe', 'unsafe']
violence detected, sending frames to server
Status Code: 201
Response: {'status': 'success', 'message': 'SOS created successfully', 'sos_id': 6, 'room_id': '21d16333-928f-4a32-8a90-dfef1b9fb858'}
Status Code: 201
Response: {'status': 'succes